In [1]:
import gcsfs
import os
import pandas as pd
import netCDF4 as nc
import tempfile
import h3
import numpy as np
from dask.distributed import Client

In [2]:
import os
import pandas as pd

# Specify the directory where your parquet files are stored
parquet_directory = 'data/processed/combined_output/period=1'

# List all parquet files in the directory
parquet_files = [os.path.join(parquet_directory, f) for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

# Read all parquet files and concatenate them into a single DataFrame
df = pd.concat([pd.read_parquet(file) for file in parquet_files])

# Optionally, reset index if needed
# df.reset_index(drop=True, inplace=True)

# Now `df` contains the combined data from all parquet files


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 52150697 entries, (np.int64(612489593553420287), Timestamp('2022-01-06 05:00:00')) to (np.int64(616776937708191743), Timestamp('2022-01-10 04:00:00'))
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   precipitation  float64
dtypes: float64(1)
memory usage: 735.3 MB


In [50]:
df = pd.read_parquet('data/processed/combined_output/period=1/dcf5a441eb3a4d899858bc434d85d5a3-0.parquet')

In [51]:
df.head()

precipitation
h3_index           timestamp                         
612490315575590911 2022-01-04 23:00:00       0.000103
612490315613339647 2022-01-04 23:00:00       0.000100
612490447593406463 2022-01-04 22:00:00       0.000103
                   2022-01-04 23:00:00       0.000109
612490447612280831 2022-01-04 22:00:00       0.000101

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15151233 entries, (np.int64(612490315575590911), Timestamp('2022-01-04 23:00:00')) to (np.int64(616777006276673535), Timestamp('2022-01-04 11:00:00'))
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   precipitation  float64
dtypes: float64(1)
memory usage: 209.4 MB


In [32]:
import gcsfs
from dask.distributed import Client
import xarray as xr

# Set up GCS connection
gcs = gcsfs.GCSFileSystem()

# List all files in the GCS bucket for your dataset
gcs_path = 'gs://your-bucket/path/to/era5/files/*.nc'
file_list = gcs.glob(gcs_path)


_request non-retriable exception: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "/opt/anaconda3/envs/coiled-xarray/lib/python3.10/site-packages/gcsfs/retry.py", line 130, in retry_request
    return await func(*args, **kwargs)
  File "/opt/anaconda3/envs/coiled-xarray/lib/python3.10/site-packages/gcsfs/core.py", line 440, in _request
    validate_response(status, contents, path, args)
  File "/opt/anaconda3/envs/coiled-xarray/lib/python3.10/site-packages/gcsfs/retry.py", line 117, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401


HttpError: Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist)., 401

In [ ]:
file_list

In [ ]:
# Open files in parallel using Dask and Xarray
ds = xr.open_mfdataset(file_list, 
                       engine='netcdf4',  # or 'h5netcdf' based on your file format
                       chunks={'time': 1},  # Adjust this based on memory and data structure
                       combine='by_coords')  # Combine the files by their coordinate system
